In [1]:
# Imports
import torch
from dask_jobqueue import SLURMCluster
from distributed import Client
from cocycles_complexity import run_experiment

In [2]:
seeds = 50
nsamples = 1000

In [3]:
cluster = SLURMCluster(
    n_workers=0,
    memory="20GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10092",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

In [4]:
futures = []
for seed in range(seeds):
    f1 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "normal")
    f2 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "gamma")
    f3 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "cauchy")
    f4 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "inversegamma")
    f5 = client.submit(run_experiment, seed, N = nsamples, noise_dist = "rademacher")
    futures += [f1,f2,f3,f4,f5]

In [5]:
futures

[<Future: pending, key: run_experiment-4b4186e0457f9fecec4a80bdbcd8c410>,
 <Future: pending, key: run_experiment-585ded73a8b7187faf7c1d2ad539e2d9>,
 <Future: pending, key: run_experiment-c36964d1b2a6de63cd8b171084cec31b>,
 <Future: pending, key: run_experiment-a358e0263385f7987fc6d548156877ce>,
 <Future: pending, key: run_experiment-cf3028908cce32584013d21cff12911b>,
 <Future: pending, key: run_experiment-c8e5d4b9da84e0dbfecd1a03f24cba6e>,
 <Future: pending, key: run_experiment-a74fbe6229d2799e9ca76396c57d7f4a>,
 <Future: pending, key: run_experiment-de1174001088fe9f81dd3346172f6b1e>,
 <Future: pending, key: run_experiment-0edc586fb132b5bf100e0cea168a6505>,
 <Future: pending, key: run_experiment-5ed0f845dc3fb6c89b7e7c98f3aeea1f>,
 <Future: pending, key: run_experiment-a4159ecab2d767ef2a046abd826f97f9>,
 <Future: pending, key: run_experiment-22ea676d8d3a1bd40ed2e135462eb07c>,
 <Future: pending, key: run_experiment-f3868cfac8ce0d8e7e774e5556294285>,
 <Future: pending, key: run_experiment

In [6]:
results = client.gather(futures)

In [7]:
client.close()
cluster.close()

2025-04-21 17:30:26,451 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [8]:
torch.save(f = "flow_complexity_results_n={0}_trials={1}_empiricalpx.pt".format(nsamples, seeds), obj = results)